This is a classification example to show how to use Oboe for training and testing, in the context of AutoML, i.e., do pipeline selection on the training set and then evaluate the performance of the selected model on the test set.

In [1]:
# necessary modules
import sys
import pandas as pd
import os
import time
import numpy as np
import multiprocessing

#Oboe modules; this will be simplified when Oboe becomes pip installable
automl_path = '../automl/'
sys.path.append(automl_path)
from auto_learner import AutoLearner
import util

#import scikit-learn modules
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# disable warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
#load and split dataset into training and test folds
data = load_iris()
x = np.array(data['data'])
y = np.array(data['target'])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# Example 1: a no-brainer use

In [3]:
# initialize the autolearner class
m = AutoLearner(p_type='classification', runtime_limit=100, load_imputed=True, verbose=True)

In [4]:
# fit autolearner on training set and record runtime
start = time.time()
m.fit(x_train, y_train)
elapsed_time = time.time() - start


Shape of training dataset: 120 data points, 4 features

Single round runtime target: 1
Fitting AutoLearner with maximum runtime 1 seconds
69
Sampling 69 entries of new row...
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-valida

3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validat

3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validation completed.
3-fold cross-validat

Pipeline fitting completed.

AutoLearner fitting complete.

Got a new ensemble in the round with rumtime target 4 seconds

Single round runtime target: 8
Fitting AutoLearner with maximum runtime 8 seconds


In [9]:
# use the fitted autolearner for prediction on test set
y_predicted = m.predict(x_test)
print("prediction error: {}".format(util.error(y_test, y_predicted, 'classification')))    
print("elapsed time: {}".format(elapsed_time))

prediction error: 0.08833333333333332
elapsed time: 121.56043982505798


In [10]:
# get names of the selected machine learning models
m.get_models()

{'ensemble method': 'greedy selection',
 'base learners': [{'imputer': {'algorithm': 'SimpleImputer',
    'hyperparameters': {'strategy': 'mean'}},
   'encoder': {'algorithm': None},
   'standardizer': {'algorithm': 'StandardScaler', 'hyperparameters': {}},
   'dim_reducer': {'algorithm': 'VarianceThreshold', 'hyperparameters': {}},
   'estimator': {'algorithm': 'KNN',
    'hyperparameters': {'n_neighbors': 5, 'p': 2}}},
  {'imputer': {'algorithm': 'SimpleImputer',
    'hyperparameters': {'strategy': 'constant'}},
   'encoder': {'algorithm': None},
   'standardizer': {'algorithm': 'StandardScaler', 'hyperparameters': {}},
   'dim_reducer': {'algorithm': 'PCA', 'hyperparameters': {'n_components': 3}},
   'estimator': {'algorithm': 'KNN',
    'hyperparameters': {'n_neighbors': 3, 'p': 1}}},
  {'imputer': {'algorithm': 'SimpleImputer',
    'hyperparameters': {'strategy': 'constant'}},
   'encoder': {'algorithm': None},
   'standardizer': {'algorithm': 'StandardScaler', 'hyperparameters': 

The following don't work

# Example 2: build an ensemble of models

In [11]:
#experimental settings
VERBOSE = False #whether to print out information indicating current fitting progress
N_CORES = 1 #number of cores
RUNTIME_BUDGET = 15

In [12]:
#optional: limit the types of algorithms
s = ['AB', 'ExtraTrees', 'GNB', 'KNN', 'RF', 'DT']

In [13]:
#autolearner arguments
autolearner_kwargs = {
    'p_type': 'classification',
    'runtime_limit': RUNTIME_BUDGET,
    'verbose': VERBOSE,
    'selection_method': 'min_variance',
    'algorithms': s,
    'stacking_alg': 'greedy',
    'n_cores': N_CORES,
    'build_ensemble': True,
}

In [14]:
#intialize the autolearner class
m = AutoLearner(**autolearner_kwargs)

In [15]:
# fit autolearner on training set and record runtime
start = time.time()
m.fit(x_train, y_train)
elapsed_time = time.time() - start

KeyboardInterrupt: 

In [ ]:
# use the fitted autolearner for prediction on test set
y_predicted = m.predict(x_test)
print("prediction error: {}".format(util.error(y_test, y_predicted, 'classification')))
print("elapsed time: {}".format(elapsed_time))
print("individual accuracies of selected models: {}".format(m.get_model_accuracy(y_test)))

In [ ]:
# get names of the selected machine learning models
m.get_models()

# Example 3: just select a collection of promising models without building an ensemble afterwards

In [ ]:
#experimental settings
VERBOSE = False #whether to print out information indicating current fitting progress
N_CORES = 1 #number of cores
RUNTIME_BUDGET = 15

In [ ]:
#optional: limit the types of algorithms
s = ['AB', 'ExtraTrees', 'GNB', 'KNN', 'RF', 'DT']

In [ ]:
#autolearner arguments
autolearner_kwargs = {
    'p_type': 'classification',
    'runtime_limit': RUNTIME_BUDGET,
    'verbose': VERBOSE,
    'selection_method': 'min_variance',
    'algorithms': s,
    'stacking_alg': 'greedy',
    'n_cores': N_CORES,
    'build_ensemble': False,
}

In [ ]:
#intialize the autolearner class
m = AutoLearner(**autolearner_kwargs)

In [ ]:
# fit autolearner on training set and record runtime
start = time.time()
m.fit(x_train, y_train)
elapsed_time = time.time() - start

In [ ]:
# use the fitted autolearner for prediction on test set
y_predicted = m.predict(x_test)
 
print("elapsed time: {}".format(elapsed_time))
print("accuracies of selected models: {}".format(m.get_model_accuracy(y_test)))

Note that we do not have a single accuracy value here if we do not build an ensemble, instead, we just have a collection of fitted models with individual accuracies reported.

In [ ]:
# get names of the selected machine learning models
m.get_models()